In [3]:
import pandas as pd
file = pd.read_excel('Job Suggestions.xlsx')
file.to_csv('jo.csv', index=True, header=None)

df_all = pd.read_csv('jo.csv')
df_all.columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
df = df_all[['A','G', 'H', 'I']]
# df.shape    # (634, 1)
# df.head()

# combining columns G, H, I
df["text"] = df["G"].astype(str) + " " + df["H"].astype(str) + " " + df["I"].astype(str)
# df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [5]:
# data preprocessing

import re
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = re.sub(",", "", text)    # removing punctuations
    text = re.sub("/", " ", text)
    text = re.sub("-", " ", text)
    text=re.sub("(\\d|\\W)+"," ",text)
    text_tokens = word_tokenize(text.lower())    # tokenising
    filtered_text = [w for w in text_tokens if w not in stop_words]    # removing stop_wordss
    
    # lemmatizing
    clean = [] 
    for word in filtered_text:
        clean.append(lemmatizer.lemmatize(word))# default pos_tagging is nounn, to do for verb add param as pos="v"
    
    return " ".join(clean)

df['text'] = df['text'].apply(clean_text)
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,A,G,H,I,text
0,1,Computer Software,Mobile Application Developer,7C Studio is looking for a Senior Android/iOS ...,computer software mobile application developer...
1,2,"Internet, Marketing and Advertising",Junior Web Developer,"Candidate be involved in development, testing,...",internet marketing advertising junior web deve...
2,3,Computer Software,Mobile Application Developer,7C Studio is looking for a Senior Android/iOS ...,computer software mobile application developer...
3,4,"Internet, Marketing and Advertising",Junior Web Developer,"Candidate be involved in development, testing,...",internet marketing advertising junior web deve...
4,5,"Information Technology and Services, Wireless,...",Web Developer,Indigo Quotient Labs is a startup in the Inter...,information technology service wireless consum...


In [7]:
# vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
trained = tfidf_vectorizer.fit_transform(df['text'])

In [16]:
# testing starts
user_file = pd.read_excel("testing.xlsx")
user_file.to_csv("testing.csv", index=True, header=None)

df_user_all = pd.read_csv("testing.csv")
df_user_all.columns = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
df_user = df_user_all.loc[df_user_all['A'] == 3]    #selecting `1` user
df_user = df_user[['A', 'G', 'H', 'I']]

df_user['text'] = df_user['G'].astype(str) + " " + df_user['H'].astype(str) + " " + df_user['I'].astype(str)
df_user['text'] = df_user['text'].apply(clean_text)
df_user.head()

,A,G,H,I,text
2,3,Computer Software,Crossover Software Architect - C# - $30/hr,"Ready to make $60,000 USD while working for a ...",computer software crossover software architect...


In [17]:
from sklearn.metrics.pairwise import cosine_similarity

tfidf_user = tfidf_vectorizer.transform(df_user['text'])
cos_similarity_user = map(lambda x: cosine_similarity(tfidf_user, x), trained)

output = list(cos_similarity_user)

In [13]:
def get_recommendation(top, df_all, scores):
    recommendation = pd.DataFrame(columns = ['A',  'G', 'score'])
    count = 0
    for i in top:
        recommendation.at[count, 'A'] = df_all['A'][i]
        recommendation.at[count, 'G'] = df_all['G'][i]
        recommendation.at[count, 'score'] =  scores[count]
        count += 1
      
    return recommendation

In [21]:
top = sorted(range(len(output)), key=lambda i: output[i][0], reverse=True)[:10]
list_scores = [output[i][0] for i in top]
get_recommendation(top, df_all, list_scores)

,A,G,score
0,221,Computer Software,[0.9853285712877865]
1,224,Computer Software,[0.9853285712877865]
2,292,Computer Software,[0.9853285712877865]
3,293,Computer Software,[0.9853285712877865]
4,364,Computer Software,[0.9689693625587266]
5,58,Computer Software,[0.9667754754030824]
6,171,Computer Software,[0.9667754754030824]
7,173,Computer Software,[0.9667754754030824]
8,402,Computer Software,[0.9667754754030824]
9,405,Computer Software,[0.9667754754030824]
